### Tensorflow 패키지 소개

In [1]:
import tensorflow as tf

### 심볼 관계 정의

In [2]:
x = tf.Variable(1.0)
y = tf.Variable(2.0)
type(x), type(y)

(tensorflow.python.ops.variables.Variable,
 tensorflow.python.ops.variables.Variable)

In [3]:
z = x + y
type(z)

tensorflow.python.framework.ops.Tensor

In [4]:
u = tf.log(z)
type(u)

tensorflow.python.framework.ops.Tensor

### 세션

In [5]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(z))
print(sess.run(u))
sess.close()

3.0
1.0986123


In [6]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(z))
    print(sess.run(u))

3.0
1.0986123


### 미분

In [7]:
f = x ** 2
fx = tf.gradients(f, [x])

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(f))
    print(sess.run(fx))

1.0
[2.0]


### 퍼셉트론 예제

In [11]:
from sklearn.datasets import load_iris
iris = load_iris()

In [13]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [15]:
idx = np.in1d(iris.target, [0, 2])
iris.target[idx]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [24]:
# iris.data.shape, 
X_data = iris.data[idx, 0:2]
y_data = (iris.target[idx] - 1.0)[:, np.newaxis]

In [28]:
np.random.seed(0)

w = tf.Variable(1e-3 * np.random.randn(2, 1))
b = tf.Variable(1e-3 * np.random.randn(1)[0])
z = tf.tanh(tf.matmul(X_data, w) + b)
w, b, z

(<tf.Variable 'Variable_4:0' shape=(2, 1) dtype=float64_ref>,
 <tf.Variable 'Variable_5:0' shape=() dtype=float64_ref>,
 <tf.Tensor 'Tanh_1:0' shape=(100, 1) dtype=float64>)

In [34]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
#     print(sess.run(w))
#     print(sess.run(b))
    print(sess.run(z).shape)

(100, 1)


In [36]:
zero = np.zeros(1, dtype=np.float64)[0]
cost = tf.reduce_sum(tf.maximum(zero, tf.multiply(-y_data, z)))
optimizer = tf.train.GradientDescentOptimizer(1e-7)
train = optimizer.minimize(cost)

cost, train

(<tf.Tensor 'Sum:0' shape=() dtype=float64>,
 <tf.Operation 'GradientDescent' type=NoOp>)

In [44]:
sess = tf.Session()

w_summary = tf.summary.histogram("w", w)
b_summary = tf.summary.scalar("b", b)
z_summary = tf.summary.histogram("z", z)
cost_summary = tf.summary.scalar("cost", cost)
merged = tf.summary.merge_all()

writer = tf.summary.FileWriter("/Users/quartz/Dropbox/TIL/11_neuralnetwork/", sess.graph)

init = tf.global_variables_initializer()
sess.run(init)

for i in range(100):
    sess.run(train)
    summary = sess.run(merged)
    writer.add_summary(summary, i)
    if i % 10 == 0:
        print("{:4d}: ".format(i), sess.run(cost))

   0:  0.5494091733308988
  10:  0.4550718147888183
  20:  0.3607278431087564
  30:  0.2663786243415365
  40:  0.17202552496054818
  50:  0.07766991175203047
  60:  0.0009515019028440593
  70:  0.0003168280919044534
  80:  0.0002921058755255818
  90:  0.00029228987473695566


In [45]:
y_pred = sess.run(tf.sign(z))
sess.close()

In [46]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_data, y_pred)

array([[49,  1],
       [ 1, 49]])

### Keras 패키지를 사용한 신경망 구현

In [47]:
%%time

from keras.datasets import mnist
(X_train0, y_train0), (X_test0, y_test0) = mnist.load_data()

Using TensorFlow backend.


CPU times: user 510 ms, sys: 570 ms, total: 1.08 s
Wall time: 1.47 s


In [52]:
print(X_train0.shape, X_train0.dtype)
print(y_train0.shape, y_train0.dtype)
print(X_test0.shape, X_test0.dtype)
print(y_test0.shape, y_test0.dtype)

(60000, 28, 28) uint8
(60000,) uint8
(10000, 28, 28) uint8
(10000,) uint8


In [56]:
X_train = X_train0.reshape(60000, 784).astype('float32') / 255.0
X_test = X_test0.reshape(10000, 784).astype('float32') / 255.0
print(X_train.shape, X_train.dtype)

(60000, 784) float32


In [59]:
# y_data one-hot encoding

from keras.utils import np_utils

Y_train = np_utils.to_categorical(y_train0, 10)
Y_test = np_utils.to_categorical(y_test0, 10)

In [60]:
np_utils.to_categorical?

Signature: np_utils.to_categorical(y, num_classes=None)
Docstring:
Converts a class vector (integers) to binary class matrix.

E.g. for use with categorical_crossentropy.

# Arguments
    y: class vector to be converted into a matrix
        (integers from 0 to num_classes).
    num_classes: total number of classes.

# Returns
    A binary matrix representation of the input. The classes axis
    is placed last.
File:      /usr/local/lib/python3.6/site-packages/keras/utils/np_utils.py
Type:      function
